# Day 09

## 기초 개념

* Database


* DBMS(Database Management System)


* DBMS의 종류
  * 관계형 데이터베이스 관리 시스템 (RDBMS: Relational Database Management System)
    * SQL(Structured Query Language)
  * NoSQL DBMS


* DBMS의 구조
  * 데이터베이스(database)
  * 테이블(table)
    * 스키마(schema)
  * 레코드(record)


* SQL
  * 생성(Create): INSERT
  * 읽기(Read): SELECT
  * 갱신(Update): UPDATE
  * 삭제(Delete): DELETE


* 트랜잭션(Transaction)
  * 데이터베이스의 작업 단위
  * 커밋(commit)
    * 트랜잭션 내용을 적용
  * 롤백(rollback)
    * 과거의 상태로 되돌림


* ACID
  * 원자성(Atomicity)
    * 작업들이 부분적으로 실행되지 않고 완결되는 것을 보장
  * 일관성(Consistency)
    * 
  * Isolation
  * Durability


## PostgreSQL

### 가동

ssh 프로그램이나 docker attach 명령, 또는 jupyter notebook의 terminal을 이용하여 도커 컨테이너 콘솔로 들어간 뒤

```
sudo service postgresql start
```

### 콘솔

콘솔에서 데이터베이스르 조작하는 명령은 권한을 가진 `postgres` 라는 계정으로 실행해야 하므로 항상 `sudo -u postgres`를 붙여서 실행한다.

* 시작
```
sudo -u postgres psql
```


* 종료
```
postgres=# \q
```


* 콘솔 도움말(vi 모드)
```
postgres=# \?
```


* SQL 도움말
```
postgres=# \h
```


## 교재

* http://www.postgresqltutorial.com/

### 샘플 데이터베이스 다운로드

```
$ wget -O dvdrental.zip http://www.postgresqltutorial.com/download/dvd-rental-sample-database/?wpdmdl=969
$ unzip dvdrental.zip
```

### 샘플 데이터베이스 생성 (bash shell에서)

```
$ sudo -u postgres createdb dvdrental

$ sudo -u postgres pg_restore -d dvdrental dvdrental.tar

$ sudo -u postgres psql
psql (9.5.6)
Type "help" for help.

postgres=# \c dvdrental
You are now connected to database "dvdrental" as user "postgres".

dvdrental-# \dt
             List of relations
 Schema |     Name      | Type  |  Owner
--------+---------------+-------+----------
 public | actor         | table | postgres
 public | address       | table | postgres
 public | category      | table | postgres
 public | city          | table | postgres
 public | country       | table | postgres
 public | customer      | table | postgres
 public | film          | table | postgres
 public | film_actor    | table | postgres
 public | film_category | table | postgres
 public | inventory     | table | postgres
 public | language      | table | postgres
 public | payment       | table | postgres
 public | rental        | table | postgres
 public | staff         | table | postgres
 public | store         | table | postgres
(15 rows)


```

## 스키마 확인

```
dvdrental=# \d+ customer


                                                              Table "public.customer"
   Column    |            Type             |                           Modifiers                            | Storage  | Stats target | Description
-------------+-----------------------------+----------------------------------------------------------------+----------+--------------+-------------
 customer_id | integer                     | not null default nextval('customer_customer_id_seq'::regclass) | plain    |              |
 store_id    | smallint                    | not null                                                       | plain    |              |
 first_name  | character varying(45)       | not null                                                       | extended |              |
 last_name   | character varying(45)       | not null                                                       | extended |              |
 email       | character varying(50)       |                                                                | extended |              |
 address_id  | smallint                    | not null                                                       | plain    |              |
 activebool  | boolean                     | not null default true                                          | plain    |              |
 create_date | date                        | not null default ('now'::text)::date                           | plain    |              |
 last_update | timestamp without time zone | default now()                                                  | plain    |              |
 active      | integer                     |                                                                | plain    |              |
Indexes:
    "customer_pkey" PRIMARY KEY, btree (customer_id)
    "idx_fk_address_id" btree (address_id)
    "idx_fk_store_id" btree (store_id)
    "idx_last_name" btree (last_name)
Foreign-key constraints:
    "customer_address_id_fkey" FOREIGN KEY (address_id) REFERENCES address(address_id) ON UPDATE CASCADE ON DELETE RESTRICT
Referenced by:
    TABLE "payment" CONSTRAINT "payment_customer_id_fkey" FOREIGN KEY (customer_id) REFERENCES customer(customer_id) ON UPDATE CASCADE ON DELETE RESTRICT
    TABLE "rental" CONSTRAINT "rental_customer_id_fkey" FOREIGN KEY (customer_id) REFERENCES customer(customer_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON customer FOR EACH ROW EXECUTE PROCEDURE last_updated()


```

```
dvdrental=# \d+ payment


                                                              Table "public.payment"
    Column    |            Type             |                          Modifiers                           | Storage | Stats target | Description
--------------+-----------------------------+--------------------------------------------------------------+---------+--------------+-------------
 payment_id   | integer                     | not null default nextval('payment_payment_id_seq'::regclass) | plain   |              |
 customer_id  | smallint                    | not null                                                     | plain   |              |
 staff_id     | smallint                    | not null                                                     | plain   |              |
 rental_id    | integer                     | not null                                                     | plain   |              |
 amount       | numeric(5,2)                | not null                                                     | main    |              |
 payment_date | timestamp without time zone | not null                                                     | plain   |              |
Indexes:
    "payment_pkey" PRIMARY KEY, btree (payment_id)
    "idx_fk_customer_id" btree (customer_id)
    "idx_fk_rental_id" btree (rental_id)
    "idx_fk_staff_id" btree (staff_id)
Foreign-key constraints:
    "payment_customer_id_fkey" FOREIGN KEY (customer_id) REFERENCES customer(customer_id) ON UPDATE CASCADE ON DELETE RESTRICT
    "payment_rental_id_fkey" FOREIGN KEY (rental_id) REFERENCES rental(rental_id) ON UPDATE CASCADE ON DELETE SET NULL
    "payment_staff_id_fkey" FOREIGN KEY (staff_id) REFERENCES staff(staff_id) ON UPDATE CASCADE ON DELETE RESTRICT


```

```
dvdrental=# \d+ staff



                                                             Table "public.staff"
   Column    |            Type             |                        Modifiers                         | Storage  | Stats target | Description
-------------+-----------------------------+----------------------------------------------------------+----------+--------------+-------------
 staff_id    | integer                     | not null default nextval('staff_staff_id_seq'::regclass) | plain    |              |
 first_name  | character varying(45)       | not null                                                 | extended |              |
 last_name   | character varying(45)       | not null                                                 | extended |              |
 address_id  | smallint                    | not null                                                 | plain    |              |
 email       | character varying(50)       |                                                          | extended |              |
 store_id    | smallint                    | not null                                                 | plain    |              |
 active      | boolean                     | not null default true                                    | plain    |              |
 username    | character varying(16)       | not null                                                 | extended |              |
 password    | character varying(40)       |                                                          | extended |              |
 last_update | timestamp without time zone | not null default now()                                   | plain    |              |
 picture     | bytea                       |                                                          | extended |              |
Indexes:
    "staff_pkey" PRIMARY KEY, btree (staff_id)
Foreign-key constraints:
    "staff_address_id_fkey" FOREIGN KEY (address_id) REFERENCES address(address_id) ON UPDATE CASCADE ON DELETE RESTRICT
Referenced by:
    TABLE "payment" CONSTRAINT "payment_staff_id_fkey" FOREIGN KEY (staff_id) REFERENCES staff(staff_id) ON UPDATE CASCADE ON DELETE RESTRICT
    TABLE "rental" CONSTRAINT "rental_staff_id_key" FOREIGN KEY (staff_id) REFERENCES staff(staff_id)
    TABLE "store" CONSTRAINT "store_manager_staff_id_fkey" FOREIGN KEY (manager_staff_id) REFERENCES staff(staff_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON staff FOR EACH ROW EXECUTE PROCEDURE last_updated()



```

## SQL SELECT 명령

* http://www.postgresqltutorial.com/postgresql-select/

```
SELECT * FROM customer;
```

vi 모드에서 나갈 때는 <kbd>ESC</kbd> + <kbd>q</kbd>

```
SELECT first_name,
 last_name,
 email
FROM customer;
```

## SQL WHERE 명령

* http://www.postgresqltutorial.com/postgresql-where/

```
SELECT last_name, first_name
FROM customer
WHERE first_name = 'Jamie';
```

```
SELECT last_name, first_name
FROM customer
WHERE first_name = 'Jamie' AND
 last_name = 'Rice';
```

```
SELECT customer_id,
 amount,
 payment_date
FROM payment
WHERE amount <= 1 OR amount >= 8;
```

## SQL JOIN

* http://www.postgresqltutorial.com/postgresql-inner-join/

```
SELECT
 customer.customer_id,
 first_name,
 last_name,
 email,
 amount,
 payment_date
FROM
 customer
INNER JOIN payment ON payment.customer_id = customer.customer_id;
```

```
SELECT
 customer.customer_id,
 first_name,
 last_name,
 email,
 amount,
 payment_date
FROM
 customer
INNER JOIN payment ON payment.customer_id = customer.customer_id
ORDER BY
 customer.customer_id;
 ```

```
SELECT
 customer.customer_id,
 first_name,
 last_name,
 email,
 amount,
 payment_date
FROM
 customer
INNER JOIN payment ON payment.customer_id = customer.customer_id
WHERE
 customer.customer_id = 2;
 ```

```
SELECT
 customer.customer_id,
 customer.first_name customer_first_name,
 customer.last_name customer_last_name,
 customer.email,
 staff.first_name staff_first_name,
 staff.last_name staff_last_name,
 amount,
 payment_date
FROM
 customer
INNER JOIN payment ON payment.customer_id = customer.customer_id
INNER JOIN staff ON payment.staff_id = staff.staff_id;
```

## SQL GROUPBY 명령

* http://www.postgresqltutorial.com/postgresql-group-by/

```
SELECT
 customer_id
FROM
 payment
GROUP BY
 customer_id;
```

```
SELECT
 customer_id,
 SUM (amount)
FROM
 payment
GROUP BY
 customer_id;
```

```
SELECT
 customer_id,
 SUM (amount)
FROM
 payment
GROUP BY
 customer_id
ORDER BY
 SUM (amount) DESC;
```

```
SELECT
 staff_id,
 COUNT (payment_id)
FROM
 payment
GROUP BY
 staff_id;
```

# Pandas 시계열 자료 다루기

In [7]:
import pandas as pd
import numpy as np

pandas에서 일반적인 테이블 형태의 자료와 시계열 자료의 차이점은 인덱스(Index)에 있다.

일반적인 테이블 형태의 자료는 임의의 값을 인덱스로 가질 수 있지만 시계열 자료는 다음 클래스를 인덱스로 가진다.

* DatetimeIndex : 타임스탬프

## DatatimeIndex

DatetimeIndex는 특정한 순간에 기록된 타임스탬프(timestamp) 형식의 시계열 자료를 다루기 위한 인덱스이다. 타임스탬프 인덱스는 반드시 일정한 간격으로 자료가 있어야 한다는 조건은 없다.

DatetimeIndex 타입의 인덱스는 보통 다음 방법으로 생성한다.

* pd.to_datetime 함수
* pd.date_range 함수

In [5]:
date_str = ["2016, 1, 1", "2016, 1, 4", "2016, 1, 5", "2016, 1, 6"]
idx = pd.to_datetime(date_str)
idx

DatetimeIndex(['2016-01-01', '2016-01-04', '2016-01-05', '2016-01-06'], dtype='datetime64[ns]', freq=None)

In [8]:
np.random.seed(0)
s = pd.Series(np.random.randn(4), index=idx)
s

2016-01-01    1.764052
2016-01-04    0.400157
2016-01-05    0.978738
2016-01-06    2.240893
dtype: float64

## date_range

* 시작일과 종료일 또는 시작일과 기간을 입력하면 범위 내의 날짜 및 시간 인덱스 생성
* freq 인수로 빈도 지정 가능
http://pandas.pydata.org/pandas-docs/stable/timeseries.html#dateoffset-objects

In [10]:
pd.date_range("2016-4-1", "2016-4-30")

DatetimeIndex(['2016-04-01', '2016-04-02', '2016-04-03', '2016-04-04',
               '2016-04-05', '2016-04-06', '2016-04-07', '2016-04-08',
               '2016-04-09', '2016-04-10', '2016-04-11', '2016-04-12',
               '2016-04-13', '2016-04-14', '2016-04-15', '2016-04-16',
               '2016-04-17', '2016-04-18', '2016-04-19', '2016-04-20',
               '2016-04-21', '2016-04-22', '2016-04-23', '2016-04-24',
               '2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
               '2016-04-29', '2016-04-30'],
              dtype='datetime64[ns]', freq='D')

In [11]:
pd.date_range(start="2016-4-1", periods=30)

DatetimeIndex(['2016-04-01', '2016-04-02', '2016-04-03', '2016-04-04',
               '2016-04-05', '2016-04-06', '2016-04-07', '2016-04-08',
               '2016-04-09', '2016-04-10', '2016-04-11', '2016-04-12',
               '2016-04-13', '2016-04-14', '2016-04-15', '2016-04-16',
               '2016-04-17', '2016-04-18', '2016-04-19', '2016-04-20',
               '2016-04-21', '2016-04-22', '2016-04-23', '2016-04-24',
               '2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
               '2016-04-29', '2016-04-30'],
              dtype='datetime64[ns]', freq='D')

In [12]:
pd.date_range("2016-4-1", "2016-4-30", freq="B")

DatetimeIndex(['2016-04-01', '2016-04-04', '2016-04-05', '2016-04-06',
               '2016-04-07', '2016-04-08', '2016-04-11', '2016-04-12',
               '2016-04-13', '2016-04-14', '2016-04-15', '2016-04-18',
               '2016-04-19', '2016-04-20', '2016-04-21', '2016-04-22',
               '2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
               '2016-04-29'],
              dtype='datetime64[ns]', freq='B')

In [13]:
pd.date_range("2016-4-1", "2016-12-31", freq="MS")

DatetimeIndex(['2016-04-01', '2016-05-01', '2016-06-01', '2016-07-01',
               '2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01',
               '2016-12-01'],
              dtype='datetime64[ns]', freq='MS')

In [14]:
pd.date_range("2016-4-1", "2016-12-31", freq="M")

DatetimeIndex(['2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31',
               '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30',
               '2016-12-31'],
              dtype='datetime64[ns]', freq='M')

In [15]:
pd.date_range("2016-4-1", "2016-12-31", freq="BMS")

DatetimeIndex(['2016-04-01', '2016-05-02', '2016-06-01', '2016-07-01',
               '2016-08-01', '2016-09-01', '2016-10-03', '2016-11-01',
               '2016-12-01'],
              dtype='datetime64[ns]', freq='BMS')

In [16]:
pd.date_range("2016-4-1", "2016-12-31", freq="BM")

DatetimeIndex(['2016-04-29', '2016-05-31', '2016-06-30', '2016-07-29',
               '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30',
               '2016-12-30'],
              dtype='datetime64[ns]', freq='BM')

In [17]:
pd.date_range("2016-1-1", "2016-12-31", freq="W-MON")

DatetimeIndex(['2016-01-04', '2016-01-11', '2016-01-18', '2016-01-25',
               '2016-02-01', '2016-02-08', '2016-02-15', '2016-02-22',
               '2016-02-29', '2016-03-07', '2016-03-14', '2016-03-21',
               '2016-03-28', '2016-04-04', '2016-04-11', '2016-04-18',
               '2016-04-25', '2016-05-02', '2016-05-09', '2016-05-16',
               '2016-05-23', '2016-05-30', '2016-06-06', '2016-06-13',
               '2016-06-20', '2016-06-27', '2016-07-04', '2016-07-11',
               '2016-07-18', '2016-07-25', '2016-08-01', '2016-08-08',
               '2016-08-15', '2016-08-22', '2016-08-29', '2016-09-05',
               '2016-09-12', '2016-09-19', '2016-09-26', '2016-10-03',
               '2016-10-10', '2016-10-17', '2016-10-24', '2016-10-31',
               '2016-11-07', '2016-11-14', '2016-11-21', '2016-11-28',
               '2016-12-05', '2016-12-12', '2016-12-19', '2016-12-26'],
              dtype='datetime64[ns]', freq='W-MON')

In [18]:
pd.date_range("2016-1-1", "2016-12-31", freq="WOM-2THU")

DatetimeIndex(['2016-01-14', '2016-02-11', '2016-03-10', '2016-04-14',
               '2016-05-12', '2016-06-09', '2016-07-14', '2016-08-11',
               '2016-09-08', '2016-10-13', '2016-11-10', '2016-12-08'],
              dtype='datetime64[ns]', freq='WOM-2THU')

In [19]:
pd.date_range("2016-1-1", "2016-12-31", freq="Q-DEC")

DatetimeIndex(['2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31'], dtype='datetime64[ns]', freq='Q-DEC')

# shift 연산

In [20]:
ts = pd.Series(np.random.randn(4), index=pd.date_range("2000-1-1", periods=4, freq="M"))
ts

2000-01-31    1.867558
2000-02-29   -0.977278
2000-03-31    0.950088
2000-04-30   -0.151357
Freq: M, dtype: float64

In [21]:
ts.shift(1)

2000-01-31         NaN
2000-02-29    1.867558
2000-03-31   -0.977278
2000-04-30    0.950088
Freq: M, dtype: float64

In [22]:
ts.shift(-1)

2000-01-31   -0.977278
2000-02-29    0.950088
2000-03-31   -0.151357
2000-04-30         NaN
Freq: M, dtype: float64

In [24]:
ts.shift(1, freq="M")

2000-02-29    1.867558
2000-03-31   -0.977278
2000-04-30    0.950088
2000-05-31   -0.151357
Freq: M, dtype: float64

In [25]:
ts.shift(1, freq="W")

2000-02-06    1.867558
2000-03-05   -0.977278
2000-04-02    0.950088
2000-05-07   -0.151357
Freq: WOM-1SUN, dtype: float64

## 리샘플링 (Resampling)

* up-sampling : 구간이 작아지는 경우
* down-sampling: 구간이 커지는 경우

In [26]:
ts = pd.Series(np.random.randn(100), index=pd.date_range("2000-1-1", periods=100, freq="D"))
ts.tail(20)

2000-03-21   -1.070753
2000-03-22    1.054452
2000-03-23   -0.403177
2000-03-24    1.222445
2000-03-25    0.208275
2000-03-26    0.976639
2000-03-27    0.356366
2000-03-28    0.706573
2000-03-29    0.010500
2000-03-30    1.785870
2000-03-31    0.126912
2000-04-01    0.401989
2000-04-02    1.883151
2000-04-03   -1.347759
2000-04-04   -1.270485
2000-04-05    0.969397
2000-04-06   -1.173123
2000-04-07    1.943621
2000-04-08   -0.413619
2000-04-09   -0.747455
Freq: D, dtype: float64

In [27]:
ts.resample('W').mean()

2000-01-02    0.153690
2000-01-09    0.678949
2000-01-16   -0.360469
2000-01-23    0.547293
2000-01-30   -0.035616
2000-02-06   -0.489050
2000-02-13   -0.464083
2000-02-20   -0.222374
2000-02-27   -0.594077
2000-03-05   -0.003614
2000-03-12   -0.460333
2000-03-19    0.461145
2000-03-26    0.258279
2000-04-02    0.753052
2000-04-09   -0.291346
Freq: W-SUN, dtype: float64

In [28]:
ts.resample('M').first()

2000-01-31   -0.103219
2000-02-29   -0.302303
2000-03-31   -0.907298
2000-04-30    0.401989
Freq: M, dtype: float64

In [29]:
ts = pd.Series(np.random.randn(60), index=pd.date_range("2000-1-1", periods=60, freq="T"))
ts.head(20)

2000-01-01 00:00:00    1.922942
2000-01-01 00:01:00    1.480515
2000-01-01 00:02:00    1.867559
2000-01-01 00:03:00    0.906045
2000-01-01 00:04:00   -0.861226
2000-01-01 00:05:00    1.910065
2000-01-01 00:06:00   -0.268003
2000-01-01 00:07:00    0.802456
2000-01-01 00:08:00    0.947252
2000-01-01 00:09:00   -0.155010
2000-01-01 00:10:00    0.614079
2000-01-01 00:11:00    0.922207
2000-01-01 00:12:00    0.376426
2000-01-01 00:13:00   -1.099401
2000-01-01 00:14:00    0.298238
2000-01-01 00:15:00    1.326386
2000-01-01 00:16:00   -0.694568
2000-01-01 00:17:00   -0.149635
2000-01-01 00:18:00   -0.435154
2000-01-01 00:19:00    1.849264
Freq: T, dtype: float64

In [30]:
ts.resample('10min').sum()

2000-01-01 00:00:00    8.552595
2000-01-01 00:10:00    3.007843
2000-01-01 00:20:00    0.615467
2000-01-01 00:30:00    2.584603
2000-01-01 00:40:00   -2.418811
2000-01-01 00:50:00   -2.042876
Freq: 10T, dtype: float64

In [31]:
ts.resample('10min', closed="right").sum()

1999-12-31 23:50:00    1.922942
2000-01-01 00:00:00    7.243732
2000-01-01 00:10:00    3.066058
2000-01-01 00:20:00    0.339179
2000-01-01 00:30:00    0.872688
2000-01-01 00:40:00   -2.250372
2000-01-01 00:50:00   -0.895407
Freq: 10T, dtype: float64

In [33]:
ts.resample('5min').ohlc()

,open,high,low,close
2000-01-01 00:00:00,1.922942,1.922942,-0.861226,-0.861226
2000-01-01 00:05:00,1.910065,1.910065,-0.268003,-0.155010
2000-01-01 00:10:00,0.614079,0.922207,-1.099401,0.298238
2000-01-01 00:15:00,1.326386,1.849264,-0.694568,1.849264
2000-01-01 00:20:00,0.672295,0.672295,-0.769916,-0.674333
2000-01-01 00:25:00,0.031831,0.676433,-0.635846,-0.208299
2000-01-01 00:30:00,0.396007,0.439392,-1.491258,0.166673
2000-01-01 00:35:00,0.635031,2.383145,-0.912822,1.117016
2000-01-01 00:40:00,-1.315907,1.713343,-1.315907,-0.744755
2000-01-01 00:45:00,-0.826439,1.126636,-1.079932,-1.079932


In [34]:
ts.resample('30s').ffill().head(20)

2000-01-01 00:00:00    1.922942
2000-01-01 00:00:30    1.922942
2000-01-01 00:01:00    1.480515
2000-01-01 00:01:30    1.480515
2000-01-01 00:02:00    1.867559
2000-01-01 00:02:30    1.867559
2000-01-01 00:03:00    0.906045
2000-01-01 00:03:30    0.906045
2000-01-01 00:04:00   -0.861226
2000-01-01 00:04:30   -0.861226
2000-01-01 00:05:00    1.910065
2000-01-01 00:05:30    1.910065
2000-01-01 00:06:00   -0.268003
2000-01-01 00:06:30   -0.268003
2000-01-01 00:07:00    0.802456
2000-01-01 00:07:30    0.802456
2000-01-01 00:08:00    0.947252
2000-01-01 00:08:30    0.947252
2000-01-01 00:09:00   -0.155010
2000-01-01 00:09:30   -0.155010
Freq: 30S, dtype: float64

In [36]:
ts.resample('30s').bfill().head(20)

2000-01-01 00:00:00    1.922942
2000-01-01 00:00:30    1.480515
2000-01-01 00:01:00    1.480515
2000-01-01 00:01:30    1.867559
2000-01-01 00:02:00    1.867559
2000-01-01 00:02:30    0.906045
2000-01-01 00:03:00    0.906045
2000-01-01 00:03:30   -0.861226
2000-01-01 00:04:00   -0.861226
2000-01-01 00:04:30    1.910065
2000-01-01 00:05:00    1.910065
2000-01-01 00:05:30   -0.268003
2000-01-01 00:06:00   -0.268003
2000-01-01 00:06:30    0.802456
2000-01-01 00:07:00    0.802456
2000-01-01 00:07:30    0.947252
2000-01-01 00:08:00    0.947252
2000-01-01 00:08:30   -0.155010
2000-01-01 00:09:00   -0.155010
2000-01-01 00:09:30    0.614079
Freq: 30S, dtype: float64

**인과관계를 감안 backward fill은 잘 사용하지 않는다.**

# Dask 사용법 기초

원래 Pandas 는 데이터베이스나 CSV 파일의 데이터를 모두 메모리로 읽어들인 다음 메모리 위에서 데이터를 처리한다. 하지만 데이터의 양이 많은 경우에는 메모리의 제한으로 데이터프레임을 만들 수 없는 경우가 있다. 또한 데이터프레임의 크기가 너무 크면 질의나 그룹 연산을 할 때 하나의 CPU 코어로 처리하기에는 시간이 너무 많이 걸릴 수도 있다. 

이러한 경우에 도움이 되는 것이 Dask 패키지이다. Dask 패키지는 Pandas 데이터프레임 형식으로 빅데이터를 처리하기 위한 파이썬 패키지로 다음과 같은 두 가지 기능을 가진다.

1. 가상 데이터프레임
1. 병렬처리용 작업 스케줄러


In [39]:
import dask.dataframe as dd

In [41]:
df = dd.read_csv("tips.csv")
df

,total_bill,tip,sex,smoker,day,time,size
npartitions=1,,,,,,,
None,float64,float64,object,object,object,object,int64
None,...,...,...,...,...,...,...


`read_csv` 명령으로 데이터 파일에 대한 가상 데이터프레임 `df`를 만든다.

`df`는 데이터프레임과 유사하지만 실제로 데이터를 메모리에 읽지 않았기 때문에 값은 표시되지 않는다.  

`head`, `tail` 명령을 내리면 그 때서야 일부 데이터를 읽어서 표시한다.

In [42]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [43]:
df.tip.mean()

dd.Scalar<series-..., dtype=float64>

데이터프레임과 달리 바로 결과가 나오지 않는다. 그 이유는 연산 반환값이 결과가 아닌 작업(task)이기 때문이다. 구체적으로 어떤 작업인지를 보려면 visualize 메서드를 사용하여 작업 그래프(graph)를 볼 수 있다. 작업 그래프란 이 계산을 하기 위해 실제로 CPU가 해야 할 일들의 순서도라고 볼 수 있다.

In [44]:
df.tip.mean().compute()

2.9982786885245902

In [46]:
(df.tip / df.total_bill).compute()

0      0.059447
1      0.160542
2      0.166587
3      0.139780
4      0.146808
5      0.186240
6      0.228050
7      0.116071
8      0.130319
9      0.218539
10     0.166504
11     0.141804
12     0.101816
13     0.162778
14     0.203641
15     0.181650
16     0.161665
17     0.227747
18     0.206246
19     0.162228
20     0.227679
21     0.135535
22     0.141408
23     0.192288
24     0.160444
25     0.131387
26     0.149589
27     0.157604
28     0.198157
29     0.152672
         ...   
214    0.230742
215    0.085271
216    0.106572
217    0.129422
218    0.186047
219    0.102522
220    0.180921
221    0.259314
222    0.223776
223    0.187735
224    0.117735
225    0.153657
226    0.198216
227    0.146699
228    0.204819
229    0.130199
230    0.083299
231    0.191205
232    0.291990
233    0.136490
234    0.193175
235    0.124131
236    0.079365
237    0.035638
238    0.130338
239    0.203927
240    0.073584
241    0.088222
242    0.098204
243    0.159744
Length: 244, dtype: floa

In [55]:
df['tip_pct'] = (df.tip / df.total_bill)
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


# consumer data: dask 연습

Dask의 가상 데이터프레임이므로 원천 데이터 파일을 하나가 아닌 복수로 설정할 수도 있다. 예를 들어 앞서 보았던 `data1.csv` 파일 이외에도 다음과 같이 `data2.csv', 'data3.csv` 파일이 있을 경우, 이 파일을 한 번에 하나의 데이터프레임으로 읽어들일 수도 있다.

In [72]:
%%writefile data2.csv
time,temperature,humidity
0,22,58
1,21,57
2,25,57
3,26,55
4,22,53
5,23,59

Writing data2.csv


In [73]:
%%writefile data3.csv
time,temperature,humidity
0,22,58
1,21,57
2,25,57
3,26,55
4,22,53
5,23,59

Writing data3.csv


In [74]:
df = dd.read_csv("data*.csv")

In [75]:
df.count().compute()

time           12
temperature    12
humidity       12
dtype: int64

In [77]:
df.temperature.describe().compute()

count    12.000000
mean     23.166667
std       1.850471
min      21.000000
25%      22.000000
50%      22.500000
75%      24.500000
max      26.000000
dtype: float64